In [239]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import config
import os.path
from os import path

from create_db import create_connection, create_table, clean_column_names
from populate_db import extract_data, insert_records
import query_db as qd

import importlib

In [240]:
importlib.reload(qd)

<module 'query_db' from '/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/query_db.py'>

In [241]:
import datetime
import re
import numpy as np

In [242]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [243]:
#coded_offenses = pd.read_excel('https://github.com/christi-liongson/covid_decarceration/blob/construct_public_safety_data/data/Coding%20Offenses%20-%20For%20GitHub.xlsx',sheet_name="Coding - FINAL")
coded_offenses = pd.read_excel('../data/Coding Offenses - For GitHub.xlsx',sheet_name="Coding - FINAL")

In [244]:
coded_offenses.head()

,Primary offense code,Description (if needed),Decided Category,Needed a check?,More lenient,More harsh
0,DRIV LICENSE REVOKED,0,1,NO,1,1
1,LARCENY,0,2,YES,1,3
2,DWI DRIVING WHILE IMPAIRED,0,2,NO,2,2
3,FELONY B&E,"Felony Breaking and Entering, as opposed to Mi...",3,YES,2,4
4,WORTHLESS CHECK,0,1,NO,1,1


In [245]:
start = datetime.datetime.now()
query_court_commitment = '''
                        SELECT A.OFFENDER_NC_DOC_ID_NUMBER as ID, 
                            A.COMMITMENT_PREFIX, 
                            A.EARLIEST_SENTENCE_EFFECTIVE_DT, 
                            A.MOST_SERIOUS_OFFENSE_CODE                              
                        FROM OFNT3BB1 A
                        WHERE NEW_PERIOD_OF_INCARCERATION_FL = "Y";
                        '''

conn = create_connection(config.database_name)
court_small = qd.query_db_notebook(conn,query_court_commitment)


query_sentence_comp = '''
                            SELECT INMATE_DOC_NUMBER as ID, 
                                INMATE_COMMITMENT_PREFIX as COMMITMENT_PREFIX, 
                                INMATE_COMPUTATION_STATUS_FLAG, 
                                max(ACTUAL_SENTENCE_END_DATE) as END_DATE,
                                max(PROJECTED_RELEASE_DATE_PRD) as PROJ_END_DATE
                            FROM INMT4BB1
                            GROUP BY INMATE_DOC_NUMBER, INMATE_COMMITMENT_PREFIX;
                        '''

sentence_compute_small = qd.query_db_notebook(conn,query_sentence_comp)


query_inmt_profile = '''
                    SELECT 
                        INMATE_DOC_NUMBER as ID,
                        INMATE_RECORD_STATUS_CODE,
                        INMATE_ADMIN_STATUS_CODE,
                        DATE_OF_LAST_INMATE_MOVEMENT,
                        TYPE_OF_LAST_INMATE_MOVEMENT,
                        CURRENT_COMMITMENT_PREFIX,
                        INMATE_GENDER_CODE as GENDER,
                        INMATE_RACE_CODE as RACE,
                        INMATE_BIRTH_DATE as BIRTH_DATE,
                        INMATE_ETHNIC_AFFILIATION as ETHNICITY,
                        INMATE_CONTROL_STATUS_CODE as CONTROL_STATUS,
                        INMATE_SPECIAL_CHARACTERISTICS as SPECIAL_CHARS,
                        TOTAL_DISCIPLINE_INFRACTIONS,
                        LATEST_DISCIPLINE_INFRACTION,
                        LAST_DISCIPLINE_INFRACTION_DT
                    FROM INMT4AA1;
                    '''

query_inmt_profile = '''
                    SELECT 
                        INMATE_DOC_NUMBER as ID,
                        INMATE_RECORD_STATUS_CODE,
                        INMATE_ADMIN_STATUS_CODE,
                        DATE_OF_LAST_INMATE_MOVEMENT,
                        TYPE_OF_LAST_INMATE_MOVEMENT,
                        CURRENT_COMMITMENT_PREFIX,
                        INMATE_CONTROL_STATUS_CODE as CONTROL_STATUS
                    FROM INMT4AA1;
                    '''

inmt_profile = qd.query_db_notebook(conn,query_inmt_profile)

query_offender_profile = '''
                        SELECT 
                        OFFENDER_NC_DOC_ID_NUMBER as ID,
                        OFFENDER_GENDER_CODE as GENDER,
                        OFFENDER_RACE_CODE as RACE,
                        OFFENDER_BIRTH_DATE as BIRTH_DATE,
                        STATE_WHERE_OFFENDER_BORN as STATE_BORN,
                        OFFENDER_ETHNIC_CODE as ETHNICITY,
                        OFFENDER_CITIZENSHIP_CODE as CITIZENSHIP                        
                    FROM OFNT3AA1;
                            
                        '''

offender_profile = qd.query_db_notebook(conn,query_offender_profile)

query_disc_infraction = '''
                    SELECT INMATE_DOC_NUMBER as ID,
                            COUNT(*) as DISCIPLINARY_INFRACTION_COUNT                            
                    FROM INMT9CF1
                    GROUP BY INMATE_DOC_NUMBER;
                    '''


disc_infraction = qd.query_db_notebook(conn,query_disc_infraction)

conn.close

data = court_small.merge(sentence_compute_small, on=['ID','COMMITMENT_PREFIX'], how='outer')
data = data.merge(inmt_profile, on=['ID'], how = 'left')
data = data.merge(offender_profile, on=['ID'], how = 'left')
data = data.merge(disc_infraction, on=['ID'], how='left')


stop = datetime.datetime.now()
print("Time Elapsed:", stop - start) 


Time Elapsed: 0:05:27.039556


In [6]:
data.to_csv('datasetA_court_sentcomp.csv', index=False)

In [67]:
data.shape

(903181, 5)

In [95]:
data.groupby('INMATE_COMPUTATION_STATUS_FLAG').count()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,END_DATE
INMATE_COMPUTATION_STATUS_FLAG,,,,,
ACTIVE,27039,27039,27009,27009,27039
EAR.TERM,2970,2970,2897,2896,2970
EXPIRED,853716,853716,853022,530070,853716
FUTURE,6184,6184,6164,6164,6184
PAROLED,198,198,197,197,198
POST REL,11797,11797,11777,11777,11797
UNKNOWN,15,15,15,15,15


In [83]:
# Missing most serious offense code  - 324,308 - about a third of the data :(
data['MOST_SERIOUS_OFFENSE_CODE'].isnull().sum()

324308

In [96]:
# data in court commitment but not sentence computation - ~ 1,200
# I spot checked some of these and they are  ACTIVE sentences, whereas others just seem to be weird data quirks..
data[data['END_DATE'].isnull()].head()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE
114,0000156,02,1995-03-15,RIOT & CIVIL DISORDERS,NaN,NaN
287,0000327,BA,1991-05-30,None,NaN,NaN
2867,0002686,BA,1993-12-06,MISD B&E,NaN,NaN
3418,0003172,BA,1989-03-06,None,NaN,NaN
4167,0003770,BA,1995-03-25,LARCENY OF MOTOR VEHICLE,NaN,NaN


In [97]:
data[data['EARLIEST_SENTENCE_EFFECTIVE_DT'].isnull()].head()
# 838 observations that appear in sentence computation but not in court commitment
# i looked up a few of them and it seemed like their full record was unavailable? might be some very recent cases with a
# bit of a time lag. since there are only ~1000 in a dataset of 1 million, i'm not too worried about dropping these

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE
902343,0001133,BA,NaN,NaN,EXPIRED,1996-06-23
902344,0001166,AI,NaN,NaN,EXPIRED,1990-07-06
902345,0001407,BE,NaN,NaN,EXPIRED,2011-07-29
902346,0001640,BA,NaN,NaN,EXPIRED,1998-03-25
902347,0005195,BF,NaN,NaN,EXPIRED,2006-04-06


In [251]:
start = datetime.datetime.now()

query_sentence_component = '''
                            SELECT OFFENDER_NC_DOC_ID_NUMBER as ID, 
                                        COMMITMENT_PREFIX, 
                                        SENTENCE_COMPONENT_NUMBER,
                                        PRIMARY_OFFENSE_CODE,
                                        PRIMARY_FELONYMISDEMEANOR_CD,
                                        SENTENCING_PENALTY_CLASS_CODE,
                                        PRIOR_RECORD_LEVEL_CODE,
                                        MINIMUM_SENTENCE_LENGTH,
                                        MAXIMUM_SENTENCE_LENGTH,
                                        SENTENCE_TYPE_CODE,
                                        COUNTY_OF_CONVICTION_CODE
                            FROM OFNT3CE1
                            WHERE SENTENCE_TYPE_CODE LIKE '%PRISONS%';
                            '''

# query_sentence_component = '''
#                             SELECT OFFENDER_NC_DOC_ID_NUMBER as ID, 
#                                         COMMITMENT_PREFIX, 
#                                         SENTENCE_COMPONENT_NUMBER,
#                                         PRIMARY_OFFENSE_CODE,
#                                         PRIMARY_FELONYMISDEMEANOR_CD,
#                                         SENTENCING_PENALTY_CLASS_CODE,
#                                         PRIOR_RECORD_LEVEL_CODE,
#                                         MINIMUM_SENTENCE_LENGTH,
#                                         MAXIMUM_SENTENCE_LENGTH,
#                                         SENTENCE_TYPE_CODE
#                             FROM OFNT3CE1;
#                             '''

#                            WHERE SENTENCE_TYPE_CODE = 'DEPT OF CORR DIV OF PRISONS';

# query_sentence_component = '''
#                             SELECT SENTENCE_TYPE_CODE,
#                                         COUNT(SENTENCE_TYPE_CODE)
#                             FROM OFNT3CE1
#                             GROUP BY SENTENCE_TYPE_CODE;
#                             '''
# query_sentence_component = '''
#                             SELECT COUNT(*)
#                             FROM OFNT3CE1;
#                             '''


conn = create_connection(config.database_name)
sent_comp_small = qd.query_db_notebook(conn,query_sentence_component)

stop = datetime.datetime.now()
print("Time Elapsed:", stop - start) 


Time Elapsed: 0:02:18.801331


In [63]:
# Sanity check
 # I ran the query with "WHERE SENTENCE_TYPE_CODE LIKE '%PRISONS%'" to capture all sentences that resulted in
    # incarceration (not probation). this results in 1,728,836 rows.  this query runs pretty quickly. however, i wanted to sanity check that
    # this was capturing all commitment prefixes which are letters (e.g. AB, BA, etc) and not numbers. This 
    # should be true if our understanding is correct that letters correspond to incarceration and numbers
    # correspond to probation
    # running the full query takes about 8 minutes, then i restrict to commitment prefixes that are letters
    # the result is also 1,728,836, confirming that it is ok to use the first query since commitment prefixes
    # that are letters correspond to sentence type codes that result in incarceration


sent_comp_small_prefix = sent_comp_small[sent_comp_small['COMMITMENT_PREFIX'].str.contains('[a-zA-Z]')]

In [62]:
sent_comp_small_prefix.shape

(1728836, 10)

In [42]:
sent_comp_small.to_csv('datasetB_sentcomponent_only_incarcerated.csv', index=False)

In [38]:
# 4,344,946 total rows in sentence component
# 1,728,836 Dept of corr div of prisons

# query_court_commitment = '''
#                         SELECT NEW_PERIOD_OF_INCARCERATION_FL, COUNT(NEW_PERIOD_OF_INCARCERATION_FL)
#                         FROM OFNT3BB1
#                         GROUP BY NEW_PERIOD_OF_INCARCERATION_FL;
#                         '''

In [90]:
# query_sentence_computation = '''
#                             SELECT INMATE_DOC_NUMBER as ID, 
#                                    INMATE_COMMITMENT_PREFIX as COMMITMENT_PREFIX, 
#                                    INMATE_SENTENCE_COMPONENT, 
#                                    ACTUAL_SENTENCE_END_DATE
#                             FROM INMT4BB1
#                             WHERE INMATE_DOC_NUMBER = '0002686';
#                             '''

# query_sentence_computation = '''
#                             SELECT *
#                             FROM INMT4BB1
#                             WHERE INMATE_COMPUTATION_STATUS_FLAG = "ACTIVE"
#                             LIMIT 1000;
#                             '''


# query_sentence_computation = '''
#                             SELECT INMATE_COMPUTATION_STATUS_FLAG, 
#                                    COUNT(INMATE_COMPUTATION_STATUS_FLAG) 
#                             FROM INMT4BB1
#                             GROUP BY INMATE_COMPUTATION_STATUS_FLAG;
#                             '''



In [91]:
# conn = create_connection(config.database_name)
# sentence_computation2 = qd.query_db_notebook(conn,query_sentence_computation)
# conn.close

<function Connection.close>

In [28]:
# # Working query, but takes a LONG time to run. much quick to query data separately then join
# query_court_commitment = '''
#                         SELECT A.OFFENDER_NC_DOC_ID_NUMBER as ID, 
#                                 A.COMMITMENT_PREFIX, 
#                                 A.EARLIEST_SENTENCE_EFFECTIVE_DT, 
#                                 A.MOST_SERIOUS_OFFENSE_CODE, 
#                                 (SELECT max(ACTUAL_SENTENCE_END_DATE)
#                                     FROM INMT4BB1 B                                    
#                                     WHERE A.OFFENDER_NC_DOC_ID_NUMBER = B.INMATE_DOC_NUMBER
#                                     AND A.COMMITMENT_PREFIX = B.INMATE_COMMITMENT_PREFIX
#                                     GROUP BY INMATE_DOC_NUMBER, INMATE_COMMITMENT_PREFIX) as END_DATE                                
#                         FROM OFNT3BB1 A
#                         WHERE NEW_PERIOD_OF_INCARCERATION_FL = "Y"
#                         LIMIT 1000;
#                         '''

## Charmaine's WIP code

To-do:
1. Check the number of unique primary offense codes for every combination of ID and COMMITMENT_PREFIX
2. Check what is going on with the 9999999 values in MINIMUM_SENTENCE_LENGTH
3. Spot check IDs in court commitment data to compare most serious offense

In [248]:
# Import CSVs
import pandas as pd

dataset_A = pd.read_csv('../data/datasetA_court_sentcomp.csv')
dataset_B = pd.read_csv('../data/datasetB_sentcomponent_only_incarcerated.csv')

/Users/daminisharma/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
dataset_A.head()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE
0,4,AA,1983-07-12,NaN,EXPIRED,1984-07-11
1,6,AA,1973-01-30,NaN,EXPIRED,1973-03-28
2,6,AB,1973-04-11,NaN,EXPIRED,1975-08-18
3,8,AA,1990-04-09,NaN,EXPIRED,1990-05-17
4,8,AB,1993-08-30,NaN,EXPIRED,1994-01-26


In [3]:
dataset_A.columns

Index(['ID', 'COMMITMENT_PREFIX', 'EARLIEST_SENTENCE_EFFECTIVE_DT',
       'MOST_SERIOUS_OFFENSE_CODE', 'INMATE_COMPUTATION_STATUS_FLAG',
       'END_DATE'],
      dtype='object')

In [4]:
dataset_B.head()

,ID,COMMITMENT_PREFIX,SENTENCE_COMPONENT_NUMBER,PRIMARY_OFFENSE_CODE,PRIMARY_FELONYMISDEMEANOR_CD,SENTENCING_PENALTY_CLASS_CODE,PRIOR_RECORD_LEVEL_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,SENTENCE_TYPE_CODE
0,4,AA,1,SELL SCHEDULE II,FELON,CLASS H,NaN,0,30000,DEPT OF CORR DIV OF PRISONS
1,4,AA,2,SELL SCHEDULE II,FELON,CLASS H,NaN,0,30000,DEPT OF CORR DIV OF PRISONS
2,6,AA,1,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,60,90,DEPT OF CORR DIV OF PRISONS
3,6,AB,1,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
4,6,AB,2,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS


In [5]:
dataset_B.columns

Index(['ID', 'COMMITMENT_PREFIX', 'SENTENCE_COMPONENT_NUMBER',
       'PRIMARY_OFFENSE_CODE', 'PRIMARY_FELONYMISDEMEANOR_CD',
       'SENTENCING_PENALTY_CLASS_CODE', 'PRIOR_RECORD_LEVEL_CODE',
       'MINIMUM_SENTENCE_LENGTH', 'MAXIMUM_SENTENCE_LENGTH',
       'SENTENCE_TYPE_CODE'],
      dtype='object')

In [6]:
dataset_A.shape

(903181, 6)

In [7]:
dataset_B.shape

(1728836, 10)

In [8]:
# Group B by ID and COMMITMENT_PREFIX, find maximum of MINIMUM_SENTENCE_LENGTH
check_minsentence = pd.DataFrame(dataset_B.groupby(['ID', 'COMMITMENT_PREFIX','PRIMARY_FELONYMISDEMEANOR_CD', 'PRIMARY_OFFENSE_CODE'])['MINIMUM_SENTENCE_LENGTH'].max().reset_index(name='max'))
check_minsentence.head(10)

,ID,COMMITMENT_PREFIX,PRIMARY_FELONYMISDEMEANOR_CD,PRIMARY_OFFENSE_CODE,max
0,4,AA,FELON,SELL SCHEDULE II,0
1,6,AA,MISD.,WORTHLESS CHECK,60
2,6,AB,MISD.,WORTHLESS CHECK,30
3,8,AA,MISD.,DWI DRIVING WHILE IMPAIRED,10000
4,8,AB,FELON,HABITUAL IMPAIRED DRIVING,0
5,8,BA,FELON,HABITUAL IMPAIRED DRIVING,0
6,10,AA,FELON,SAFECRACKING/SAFE ROBBERY,0
7,10,AB,FELON,B & E & L,100000
8,10,AB,FELON,LARCENY OF MOTOR VEHICLE,50000
9,10,AB,FELON,SAFECRACKING/SAFE ROBBERY,200000


In [22]:
min_sentence = pd.DataFrame(dataset_B.groupby(['ID', 'COMMITMENT_PREFIX'])['MINIMUM_SENTENCE_LENGTH'].max().reset_index(name='max_min'))
min_sentence.head(10)

,ID,COMMITMENT_PREFIX,max_min
0,4,AA,0
1,6,AA,60
2,6,AB,30
3,8,AA,10000
4,8,AB,0
5,8,BA,0
6,10,AA,0
7,10,AB,200000
8,14,AA,1800
9,14,AB,600


In [10]:
# Filter dataset_B to only these rows
filter_tuples = [tuple(x) for x in min_sentence.to_numpy()]

filtered_B = dataset_B[dataset_B[['ID', 'COMMITMENT_PREFIX', 'MINIMUM_SENTENCE_LENGTH']].apply(tuple, axis=1).isin(filter_tuples)]
filtered_B.head(10)

,ID,COMMITMENT_PREFIX,SENTENCE_COMPONENT_NUMBER,PRIMARY_OFFENSE_CODE,PRIMARY_FELONYMISDEMEANOR_CD,SENTENCING_PENALTY_CLASS_CODE,PRIOR_RECORD_LEVEL_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,SENTENCE_TYPE_CODE
0,4,AA,1,SELL SCHEDULE II,FELON,CLASS H,NaN,0,30000,DEPT OF CORR DIV OF PRISONS
1,4,AA,2,SELL SCHEDULE II,FELON,CLASS H,NaN,0,30000,DEPT OF CORR DIV OF PRISONS
2,6,AA,1,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,60,90,DEPT OF CORR DIV OF PRISONS
3,6,AB,1,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
4,6,AB,2,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
5,6,AB,3,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
6,6,AB,4,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
7,6,AB,5,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
8,6,AB,6,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
9,6,AB,7,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS


Based on the sample of observations above, there are quite a few ties, but it might be that they are all of the same PRIMARY_OFFENSE_CODE (e.g., rows 3-9). I will check how many unique primary offense codes there are for every ID and COMMITMENT_PREFIX combination.

In [11]:
unique_offenses = pd.DataFrame(filtered_B.groupby(['ID', 'COMMITMENT_PREFIX'])['PRIMARY_OFFENSE_CODE'].nunique().reset_index(name='count'))
unique_offenses['count'].describe()

count    891122.000000
mean          1.199647
std           0.547120
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max          12.000000
Name: count, dtype: float64

In [19]:
unique_offenses.head(1)

,ID,COMMITMENT_PREFIX,count
0,4,AA,1


It looks like the vast majority of ID and COMMITMENT_PREFIX combinations have a single PRIMARY_OFFENSE_CODE, but there is at least one combination that had 12 unique PRIMARY_OFFENSE_CODES.

In [12]:
more_than_one_offense = unique_offenses[unique_offenses['count'] > 1].sort_values('count')
more_than_one_offense.head(10)

,ID,COMMITMENT_PREFIX,count
11,19,AA,2
482426,433706,AB,2
482420,433699,BB,2
482410,433692,BC,2
482409,433692,BB,2
482400,433688,AB,2
482391,433686,AA,2
482360,433668,AD,2
482353,433660,BA,2
482338,433641,AC,2


In [13]:
more_than_one_offense.shape

(134415, 3)

There are 134,415 observations (ID and COMMITMENT_PREFIX combinations) that have more than one unique PRIMARY_OFFENSE_CODE. Check what PRIMARY_OFFENSE_CODES are associated with these observations (in dataset B), and check (if available) what their MOST_SERIOUS_OFFENSE was (in dataset A).

In [14]:
offenses_filter = [tuple(x) for x in more_than_one_offense[['ID', 'COMMITMENT_PREFIX']].to_numpy()]

filtered_B_mt1 = filtered_B[filtered_B[['ID', 'COMMITMENT_PREFIX']].apply(tuple, axis=1).isin(offenses_filter)]
filtered_B_mt1.head()

,ID,COMMITMENT_PREFIX,SENTENCE_COMPONENT_NUMBER,PRIMARY_OFFENSE_CODE,PRIMARY_FELONYMISDEMEANOR_CD,SENTENCING_PENALTY_CLASS_CODE,PRIOR_RECORD_LEVEL_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,SENTENCE_TYPE_CODE
41,19,AA,1,MISD B&E,MISD.,MISD.(PRE-STRUCTURE),NaN,2400,2400,DEPT OF CORR DIV OF PRISONS
42,19,AA,2,LARCENY (OVER $200),MISD.,MISD.(PRE-STRUCTURE),NaN,2400,2400,DEPT OF CORR DIV OF PRISONS
45,20,BA,1,FELONY B&E,FELON,CLASS H,NaN,0,100000,DEPT OF CORR DIV OF PRISONS
46,20,BA,2,B & E & L,FELON,CLASS H,NaN,0,100000,DEPT OF CORR DIV OF PRISONS
47,20,BA,3,FELONY B&E,FELON,CLASS H,NaN,0,60000,DEPT OF CORR DIV OF PRISONS


We have some odd observations where the primary offense is a felony, but MINIMUM_SENTENCE_LENGTH is 0 and MAXIMUM_SENTENCE_LENGTH > 0 (e.g., index 45-47)

In [15]:
filtered_B_mt1 = filtered_B_mt1[['ID', 'COMMITMENT_PREFIX', 'PRIMARY_OFFENSE_CODE', 'MINIMUM_SENTENCE_LENGTH', 'MAXIMUM_SENTENCE_LENGTH']]
filtered_B_mt1

,ID,COMMITMENT_PREFIX,PRIMARY_OFFENSE_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH
41,19,AA,MISD B&E,2400,2400
42,19,AA,LARCENY (OVER $200),2400,2400
45,20,BA,FELONY B&E,0,100000
46,20,BA,B & E & L,0,100000
47,20,BA,FELONY B&E,0,60000
74,33,AD,ROBBERY W/DANGEROUS WEAPON,0,180000
75,33,AD,ESCAPE PRISON,0,30000
83,35,BA,FELONY B&E,0,80000
84,35,BA,B & E & L,0,30000
103,49,AC,INDECENT LIBERTY W/CHILD,0,60000


In [21]:
find_max_max = pd.DataFrame(filtered_B_mt1.groupby(['ID', 'COMMITMENT_PREFIX'])['MAXIMUM_SENTENCE_LENGTH'].max().reset_index(name='max_max'))
find_max_max.sample(5)

,ID,COMMITMENT_PREFIX,max_max
59641,300717,AC,20000
15793,79118,AA,30000
68125,344547,BB,10500
17307,87117,AA,30000
27050,136181,AC,20000


In [26]:
by_max_tuples = [tuple(x) for x in find_max_max.to_numpy()]
filtered_max = filtered_B_mt1[filtered_B_mt1[['ID', 'COMMITMENT_PREFIX', 'MAXIMUM_SENTENCE_LENGTH']].apply(tuple, axis=1).isin(by_max_tuples)]
filtered_max.head()

,ID,COMMITMENT_PREFIX,PRIMARY_OFFENSE_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH
41,19,AA,MISD B&E,2400,2400
42,19,AA,LARCENY (OVER $200),2400,2400
45,20,BA,FELONY B&E,0,100000
46,20,BA,B & E & L,0,100000
74,33,AD,ROBBERY W/DANGEROUS WEAPON,0,180000


In [30]:
#filtered_max
choose_random = pd.DataFrame(filtered_max.groupby(['ID', 'COMMITMENT_PREFIX']).size().reset_index(name='count'))
choose_random = choose_random[choose_random['count'] > 1][['ID', 'COMMITMENT_PREFIX']]

In [32]:
choose_random.shape

(72137, 2)

There are 72,137 ID and COMMITMENT_PREFIX combinations we have to choose randomly from. 

In [33]:
choose_random.head()

,ID,COMMITMENT_PREFIX
0,19,AA
1,20,BA
7,74,AA
8,74,AB
9,80,BA


In [16]:
# What do we do with MINIMUM_SENTENCE_LENGTH = 9999999
dataset_B['MINIMUM_SENTENCE_LENGTH'].describe()

count    1.728836e+06
mean     2.569733e+04
std      4.212079e+05
min      0.000000e+00
25%      0.000000e+00
50%      4.000000e+02
75%      1.200000e+03
max      9.999999e+06
Name: MINIMUM_SENTENCE_LENGTH, dtype: float64

#### What are the offenses associated with 9999999 minimum sentence lengths?

Based on the grouped dataframe below, most of the offenses are high-risk felonies, so likely equivalent to a life sentence.

In [17]:
pd.DataFrame(dataset_B[dataset_B['MINIMUM_SENTENCE_LENGTH'] == 9999999].groupby('PRIMARY_OFFENSE_CODE').size().reset_index(name='count'))

,PRIMARY_OFFENSE_CODE,count
0,BURGLARY 1ST DEGREE,1
1,COMMON LAW ROBBERY,1
2,MURDER FIRST DEGREE,448
3,RAPE FIRST DEGREE,9
4,SEXUAL OFFENSE 1ST DEGREE,1


I also checked the MAXIMUM_SENTENCE_LENGTH column to see what that looked like: either 9999999 or 0, so that squares with the hypothesis that these are life sentences. 

In [18]:
dataset_B[dataset_B['MINIMUM_SENTENCE_LENGTH'] == 9999999]['MAXIMUM_SENTENCE_LENGTH'].unique()

array([9999999, 0, '9999999'], dtype=object)

#### Main Takeaways:
* Choosing observations (ID and COMMITMENT_PREFIX combination) based on sentence length:
    * Choose the observation that has the maximum MINIMUM_SENTENCE_LENGTH
    * If there is a tie on MINIMUM_SENTENCE_LENGTH, select observation with the maximum MAXIMUM_SENTENCE_LENGTH 
    * If there is a tie on MAXIMUM_SENTENCE_LENGTH, then choose randomly
* Do not change 9999999 values to NA, since these likely correspond to life-sentences. Check against court commitment data.

In [63]:
# Finalize dataset_B
seed = 10000
# unique_offenses gives us the where ID+COMMITMENT_PREFIX have a unique max on MINIMUM_SENTENCE_LENGTH
# filtered_B_mt1 give us the records where ID+COMMITMENT_PREFIX does not have a unique max on MINIMUM_SENTENCE_LENGTH

# (MINIMUM_SENTENCE_LENGTH = max_min and unique_min = False and MAXIMUM_SENTENCE_LENGTH = max_max and unique_max = True) or
# (MINIMUM_SENTENCE_LENGTH = max_min and unique_min = False and 
#  MAXIMUM_SENTENCE_LENGTH = max_max and unique_max = False).sample(1, random_state=seed)


,ID,COMMITMENT_PREFIX,SENTENCE_COMPONENT_NUMBER,PRIMARY_OFFENSE_CODE,PRIMARY_FELONYMISDEMEANOR_CD,SENTENCING_PENALTY_CLASS_CODE,PRIOR_RECORD_LEVEL_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,SENTENCE_TYPE_CODE
0,4,AA,1,SELL SCHEDULE II,FELON,CLASS H,NaN,0,30000,DEPT OF CORR DIV OF PRISONS
1,4,AA,2,SELL SCHEDULE II,FELON,CLASS H,NaN,0,30000,DEPT OF CORR DIV OF PRISONS
2,6,AA,1,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,60,90,DEPT OF CORR DIV OF PRISONS
3,6,AB,1,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
4,6,AB,2,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS


In [ ]:
# Puling it all together - get most serious offense code for each id and commitment prefix
merged = dataset_A.merge(dataset_B, how='inner', on=['ID', 'COMMITMENT_PREFIX'])

In [209]:
# Dealing with only dataset A for the next several cells:
data.shape

(903181, 20)

In [257]:
data['EARLIEST_SENTENCE_EFFECTIVE_DT'].isnull().sum()

838

In [256]:
# sentences missing earliest effective date from datasetA
data['END_DATE'].isnull().sum()

1262

In [261]:
data[data['END_DATE']=='0001-01-01']['END_DATE'].count()

10110

In [263]:
#data[data['EARLIEST_SENTENCE_EFFECTIVE_DT']=='0001-01-01']['EARLIEST_SENTENCE_EFFECTIVE_DT'].count()
data_missing_start = data[data['EARLIEST_SENTENCE_EFFECTIVE_DT']=='0001-01-01']
data_missing_start.shape


(12595, 20)

In [264]:
data_merge_missing = data_missing_start.merge(sent_comp_small,  how='left',on = ['ID','COMMITMENT_PREFIX'])


In [265]:
# of the 12,595 observations with EARLIEST_SENTENCE_EFFECTIVE_DT == 0001-01-01, 
# 11,400 dont have a match in sentence component, from which we migtht be 
# able to get sentence start date. Not sure it seems worth it for 1,100 observations
data_merge_missing.isnull().sum()

ID                                    0
COMMITMENT_PREFIX                     0
EARLIEST_SENTENCE_EFFECTIVE_DT        0
MOST_SERIOUS_OFFENSE_CODE         15940
INMATE_COMPUTATION_STATUS_FLAG       32
END_DATE                             32
PROJ_END_DATE                        32
INMATE_RECORD_STATUS_CODE          1647
INMATE_ADMIN_STATUS_CODE           1647
DATE_OF_LAST_INMATE_MOVEMENT       1647
TYPE_OF_LAST_INMATE_MOVEMENT       1980
CURRENT_COMMITMENT_PREFIX          8484
CONTROL_STATUS                     1647
GENDER                                0
RACE                                  0
BIRTH_DATE                            0
STATE_BORN                          358
ETHNICITY                             0
CITIZENSHIP                           0
DISCIPLINARY_INFRACTION_COUNT      4425
SENTENCE_COMPONENT_NUMBER         11428
PRIMARY_OFFENSE_CODE              11438
PRIMARY_FELONYMISDEMEANOR_CD      11428
SENTENCING_PENALTY_CLASS_CODE     11428
PRIOR_RECORD_LEVEL_CODE           15942


In [252]:
sent_comp_small.head()

,ID,COMMITMENT_PREFIX,SENTENCE_COMPONENT_NUMBER,PRIMARY_OFFENSE_CODE,PRIMARY_FELONYMISDEMEANOR_CD,SENTENCING_PENALTY_CLASS_CODE,PRIOR_RECORD_LEVEL_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,SENTENCE_TYPE_CODE,COUNTY_OF_CONVICTION_CODE
0,0000004,AA,001,SELL SCHEDULE II,FELON,CLASS H,None,0000000,0030000,DEPT OF CORR DIV OF PRISONS,PERQUIMANS
1,0000004,AA,002,SELL SCHEDULE II,FELON,CLASS H,None,0000000,0030000,DEPT OF CORR DIV OF PRISONS,PERQUIMANS
2,0000006,AA,001,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),None,0000060,0000090,DEPT OF CORR DIV OF PRISONS,CUMBERLAND
3,0000006,AB,001,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),None,0000030,0000000,DEPT OF CORR DIV OF PRISONS,CUMBERLAND
4,0000006,AB,002,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),None,0000030,0000000,DEPT OF CORR DIV OF PRISONS,CUMBERLAND


In [266]:
# Step 1
# https://kanoki.org/2019/07/17/pandas-how-to-replace-values-based-on-conditions/
data['END_DATE'].mask(data['END_DATE'] == '0001-01-01', data['PROJ_END_DATE'], inplace=True)
data = data[data['END_DATE']!='0001-01-01']
data = data[data['EARLIEST_SENTENCE_EFFECTIVE_DT']!='0001-01-01']
data = data[data['END_DATE'].notna()]
data = data[data['EARLIEST_SENTENCE_EFFECTIVE_DT'].notna()]

In [267]:
data.shape

(888120, 20)

In [268]:
# Step 2
# write data to sqlite in memory so can query it to get next record
conn = sqlite3.connect(':memory:')
data.to_sql('data', conn, index=False)

start = datetime.datetime.now()
# https://stackoverflow.com/questions/37360901/sql-self-join-compare-current-record-with-the-record-of-the-previous-date
query_datasetA = '''
                        SELECT *, 
                        LEAD(COMMITMENT_PREFIX,1,0) OVER (
                                                    PARTITION BY ID
                                                    ORDER BY COMMITMENT_PREFIX
                                                    ) NextPrefix,
                        LEAD(EARLIEST_SENTENCE_EFFECTIVE_DT,1,0) OVER (
                                                    PARTITION BY ID
                                                    ORDER BY COMMITMENT_PREFIX
                                                    ) NextStart,
                        LEAD(MOST_SERIOUS_OFFENSE_CODE,1,0) OVER (
                                                    PARTITION BY ID
                                                    ORDER BY COMMITMENT_PREFIX
                                                    ) NextOffense                                                    
                                                    
                        FROM data ;

                        '''


dataset_flag = qd.query_db_notebook(conn,query_datasetA)
conn.close
stop = datetime.datetime.now()
print("Time Elapsed:", stop - start) 

Time Elapsed: 0:02:37.865425


In [270]:
# Define functions that fix dates
# specifically, some dates are top coded as 9999- usually for a life sentence
# this exceeds pandas' max date, so they first need to be re-top-coded, then turned into the date format
# date == 0 happens when an individual does NOT have a "next date" - these should be turned to Na
def fix_dates(data,date_var):
    data['new_col'] = data[date_var].astype(str).str[0:4].astype(int)
    data.loc[data['new_col']>2261, date_var] = '2261-01-02'
    data[date_var] = data[date_var].replace(0,np.nan)
    data.loc[data[date_var]=="0", date_var] = None
    data[date_var] = pd.to_datetime(data[date_var],format='%Y-%m-%d',errors='coerce')
    #df[date_var] = pd.to_datetime(df[date_var].str.split(n=1).str[0],format='%Y-%m-%d')
    return data


def get_recidivism_label(data,num_years=1):
    data['Time_Diff'] = pd.DatetimeIndex(data['NextStart']).year - pd.DatetimeIndex(data['END_DATE']).year
    data['Recidivate'] = np.nan
    # if NextPrefix != 0:
    data.loc[(data['NextPrefix']!=0) & (data['Time_Diff']<= num_years), 'Recidivate'] = 1
    data.loc[(data['NextPrefix']!=0) & (data['Time_Diff']> num_years), 'Recidivate'] = 0
    
    # if nextprefix = 0, inmate is inactive, and they did not die in prison 
    # (e.g. serving life sentence or  other wise) then 
    # recidivism = 0
    data.loc[(data['NextPrefix']==0) & (data['INMATE_ADMIN_STATUS_CODE']=='INACTIVE') & (data['TYPE_OF_LAST_INMATE_MOVEMENT']!='DEATH'), 'Recidivate'] = 0
    
    # if nextprefix = 0, inmate status code is not active or inactive(could be missing) and 
    # end date is not 2261-01-02 (life sentence), they were likely released from prison
    # recidivism = 0
    #data.loc[(data['NextPrefix']==0) & (data['INMATE_ADMIN_STATUS_CODE']!='ACTIVE') & (data['INMATE_ADMIN_STATUS_CODE']!='INACTIVE') & (data['END_DATE']!='2261-01-02'), 'Recidivate'] = 0
    
    
    return data



In [271]:
dataset_flag.head()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_RECORD_STATUS_CODE,INMATE_ADMIN_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,DISCIPLINARY_INFRACTION_COUNT,NextPrefix,NextStart,NextOffense
0,0000004,AA,1983-07-12,None,EXPIRED,1984-07-11,1984-07-11,INACTIVE,INACTIVE,1984-07-11,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1961-10-15,ALASKA,UNKNOWN,BORN IN U.S.,NaN,0,0,0
1,0000006,AA,1973-01-30,None,EXPIRED,1973-03-28,0001-01-01,INACTIVE,INACTIVE,1975-08-18,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1951-07-17,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,NaN,AB,1973-04-11,None
2,0000006,AB,1973-04-11,None,EXPIRED,1975-08-18,1974-08-10,INACTIVE,INACTIVE,1975-08-18,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1951-07-17,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,NaN,0,0,0
3,0000008,AA,1990-04-09,None,EXPIRED,1990-05-17,1990-10-09,INACTIVE,INACTIVE,1995-09-14,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1963-12-29,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,NaN,AB,1993-08-30,None
4,0000008,AB,1993-08-30,None,EXPIRED,1994-01-26,1994-02-18,INACTIVE,INACTIVE,1995-09-14,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1963-12-29,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,NaN,BA,1995-01-02,None


In [272]:
# Step 3. 
# call fix dates function to fix relevant dates
dataset_flag = fix_dates(dataset_flag,'EARLIEST_SENTENCE_EFFECTIVE_DT')
dataset_flag = fix_dates(dataset_flag,'END_DATE')
dataset_flag = fix_dates(dataset_flag,'NextStart')

In [273]:
# check that dates converted successfully
dataset_flag.dtypes

ID                                        object
COMMITMENT_PREFIX                         object
EARLIEST_SENTENCE_EFFECTIVE_DT    datetime64[ns]
MOST_SERIOUS_OFFENSE_CODE                 object
INMATE_COMPUTATION_STATUS_FLAG            object
END_DATE                          datetime64[ns]
PROJ_END_DATE                             object
INMATE_RECORD_STATUS_CODE                 object
INMATE_ADMIN_STATUS_CODE                  object
DATE_OF_LAST_INMATE_MOVEMENT              object
TYPE_OF_LAST_INMATE_MOVEMENT              object
CURRENT_COMMITMENT_PREFIX                 object
CONTROL_STATUS                            object
GENDER                                    object
RACE                                      object
BIRTH_DATE                                object
STATE_BORN                                object
ETHNICITY                                 object
CITIZENSHIP                               object
DISCIPLINARY_INFRACTION_COUNT            float64
NextPrefix          

In [274]:
# Step 4
# get recidivism flag - see decision rules and function above 
dataset_flag = get_recidivism_label(dataset_flag)
dataset_flag.head()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_RECORD_STATUS_CODE,INMATE_ADMIN_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,DISCIPLINARY_INFRACTION_COUNT,NextPrefix,NextStart,NextOffense,new_col,Time_Diff,Recidivate
0,0000004,AA,1983-07-12,None,EXPIRED,1984-07-11,1984-07-11,INACTIVE,INACTIVE,1984-07-11,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1961-10-15,ALASKA,UNKNOWN,BORN IN U.S.,NaN,0,NaT,0,0,NaN,0.0
1,0000006,AA,1973-01-30,None,EXPIRED,1973-03-28,0001-01-01,INACTIVE,INACTIVE,1975-08-18,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1951-07-17,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,NaN,AB,1973-04-11,None,1973,0.0,1.0
2,0000006,AB,1973-04-11,None,EXPIRED,1975-08-18,1974-08-10,INACTIVE,INACTIVE,1975-08-18,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1951-07-17,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,NaN,0,NaT,0,0,NaN,0.0
3,0000008,AA,1990-04-09,None,EXPIRED,1990-05-17,1990-10-09,INACTIVE,INACTIVE,1995-09-14,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1963-12-29,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,NaN,AB,1993-08-30,None,1993,3.0,0.0
4,0000008,AB,1993-08-30,None,EXPIRED,1994-01-26,1994-02-18,INACTIVE,INACTIVE,1995-09-14,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1963-12-29,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,NaN,BA,1995-01-02,None,1995,1.0,1.0


In [276]:
missing_recidivate = dataset_flag[dataset_flag['Recidivate'].isnull()]

In [281]:
missing_recidivate.shape

(40544, 26)

In [279]:
# 40k obs missing recidivism flag.
# 32,801 of those are active
# 4,325 of the inactive ones are those that died in prison
# so about 2,700 obs missing recidivate flag because they have no INMATE_ADMIN_STATUS_CODE
# let's look at just those
missing_recidivate.groupby('INMATE_ADMIN_STATUS_CODE').count()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_RECORD_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,DISCIPLINARY_INFRACTION_COUNT,NextPrefix,NextStart,NextOffense,new_col,Time_Diff,Recidivate
INMATE_ADMIN_STATUS_CODE,,,,,,,,,,,,,,,,,,,,,,,,,
ACTIVE,32801,32801,32801,32796,32801,32801,32801,32801,32801,27587,32729,32801,32801,32801,32801,31123,32789,32790,28291,32801,0,32801,32801,0,0
INACTIVE,4325,4325,4325,2645,4325,4325,4325,4325,4325,4325,2656,4325,4325,4325,4325,4063,4324,4324,3012,4325,0,4325,4325,0,0
TEM.ABSENT,697,697,697,696,697,697,697,697,697,142,696,697,697,697,697,637,672,672,319,697,0,697,697,0,0


In [284]:
missing_recidivate.groupby('TYPE_OF_LAST_INMATE_MOVEMENT').count()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_RECORD_STATUS_CODE,INMATE_ADMIN_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,DISCIPLINARY_INFRACTION_COUNT,NextPrefix,NextStart,NextOffense,new_col,Time_Diff,Recidivate
TYPE_OF_LAST_INMATE_MOVEMENT,,,,,,,,,,,,,,,,,,,,,,,,,
DEATH,4325,4325,4325,2645,4325,4325,4325,4325,4325,4325,2656,4325,4325,4325,4325,4063,4324,4324,3012,4325,0,4325,4325,0,0
ESCAPE FROM WRIT,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,0,2,2,0,0
ESCAPED,34,34,34,33,34,34,34,34,34,34,33,34,34,34,34,20,34,34,22,34,0,34,34,0,0
ESCAPEE IN CUSTODY,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,0,1,1,0,0
NEW ADMISSION,1109,1109,1109,1109,1109,1109,1109,1109,1109,1109,1109,1109,1109,1109,1109,1023,1100,1100,425,1109,0,1109,1109,0,0
OUT - TEMP LEAVE,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,74,100,0,100,100,0,0
RE-ADMISSION,1078,1078,1078,1077,1078,1078,1078,1078,1078,1078,1010,1078,1078,1078,1078,1066,1078,1078,848,1078,0,1078,1078,0,0
RECEIVE PSD,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,0,2,2,0,0
RECEIVED FROM,24186,24186,24186,24185,24186,24186,24186,24186,24186,24186,24183,24186,24186,24186,24186,22905,24183,24184,21323,24186,0,24186,24186,0,0


In [282]:
32801 + 4325 + 697

37823

In [283]:
40544 - 37823

2721

In [285]:
missing_recidivate_no_inmate_admin_record = missing_recidivate[missing_recidivate['INMATE_ADMIN_STATUS_CODE'].isnull()]


In [288]:
# All those missing INMATE_ADMIN_STATUS_CODE are also missing INMATE_RECORD_STATUS_CODE
missing_recidivate_no_inmate_admin_record.groupby('INMATE_RECORD_STATUS_CODE').count()


,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_ADMIN_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,DISCIPLINARY_INFRACTION_COUNT,NextPrefix,NextStart,NextOffense,new_col,Time_Diff,Recidivate
INMATE_RECORD_STATUS_CODE,,,,,,,,,,,,,,,,,,,,,,,,,


In [291]:
# on spot checking the IDs, the offenders show up as "inactive" for their parole/probation
# status, even though inmate_record_status in our data shows none
# they also have "current incarceration record not available"
# not really sure if this is just some weird data quirk 
# i'm okay with either dropping these 2k observations, or assuming they were released
# and
missing_recidivate_no_inmate_admin_record.head()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_RECORD_STATUS_CODE,INMATE_ADMIN_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,DISCIPLINARY_INFRACTION_COUNT,NextPrefix,NextStart,NextOffense,new_col,Time_Diff,Recidivate
12,0000020,BA,1990-05-31,None,EXPIRED,1994-12-02,1994-12-02,None,None,None,None,None,None,MALE,WHITE,1950-07-05,PENNSYLVANIA,EUROPEAN/N.AM./AUSTR,BORN IN U.S.,NaN,0,NaT,0,0,NaN,NaN
224,0000264,AA,1984-08-01,None,EXPIRED,1987-01-02,1987-01-02,None,None,None,None,None,None,MALE,WHITE,1967-10-29,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,43.0,0,NaT,0,0,NaN,NaN
592,0000575,AE,1994-05-03,None,EXPIRED,1994-09-23,1994-09-23,None,None,None,None,None,None,MALE,BLACK,1952-10-14,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,2.0,0,NaT,0,0,NaN,NaN
666,0000645,AA,1987-11-03,None,EXPIRED,1989-03-26,1989-03-26,None,None,None,None,None,None,MALE,BLACK,1967-12-14,NEW JERSEY,AFRICAN,BORN IN U.S.,NaN,0,NaT,0,0,NaN,NaN
681,0000662,AD,1985-09-23,None,EXPIRED,1986-04-07,1986-04-30,None,None,None,None,None,None,MALE,WHITE,1955-05-17,NORTH CAROLINA,EUROPEAN/N.AM./AUSTR,BORN IN U.S.,4.0,0,NaT,0,0,NaN,NaN


In [294]:
# sentence with max end date is also inactive on offender search
missing_recidivate_no_inmate_admin_record[missing_recidivate_no_inmate_admin_record['END_DATE']==missing_recidivate_no_inmate_admin_record['END_DATE'].max()]



,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_RECORD_STATUS_CODE,INMATE_ADMIN_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,DISCIPLINARY_INFRACTION_COUNT,NextPrefix,NextStart,NextOffense,new_col,Time_Diff,Recidivate
812096,1141078,BC,2011-07-25,TRAFFICKING SCHEDULE I,EXPIRED,2020-05-17,2020-05-17,None,None,None,None,None,None,FEMALE,WHITE,1983-04-04,NORTH CAROLINA,EUROPEAN/N.AM./AUSTR,BORN IN U.S.,93.0,0,NaT,0,0,NaN,NaN


In [220]:
dataset_flag.groupby('Recidivate').count()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_RECORD_STATUS_CODE,INMATE_ADMIN_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,DISCIPLINARY_INFRACTION_COUNT,NextPrefix,NextStart,NextOffense,new_col,Time_Diff
Recidivate,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,692709,692709,692709,452527,692709,692709,692709,688110,688110,688110,682473,519274,688110,692709,692708,692709,665212,692031,692046,408973,692709,261928,636906,692709,261928
1.0,158261,158261,158261,89784,158261,158261,158261,156644,156644,156644,153285,129531,156644,158261,158260,158261,156132,158247,158255,131796,158261,158261,102429,158261,158261


In [221]:
# Step 5
# Hold out active senteces
active_sentences = dataset_flag[(dataset_flag['INMATE_ADMIN_STATUS_CODE']=='ACTIVE') & (dataset_flag['NextPrefix']==0)]
active_sentences.shape

(32801, 26)

In [222]:
# Step 6
# drop observations with no recidivism flag (this will also drop active sentences, but we've already separated those)
dataset_flag = dataset_flag[(dataset_flag['Recidivate'].notnull())]
dataset_flag.shape

(850970, 26)

In [223]:
# Step 7
# Bring in coded offenses - sanity check

# this merges our coded offenses onto "most serious offense" to check how much coverage
# our variable is giving us. however, this not what we ultimately want - in the end, we want
# our codes to be merged onto "nextOffense" - i.e., the offense code for the next offense 
# someone committed that resulted in re-incarceration
# NextOffense can be missing for 2 reasons: because most serious offense is missing, or because
# the individual did not recidivate. after merging our codes onto "NextOffense", we can replace
# "Decided Category" with 0 if recidivism = 0, and leave it as NA otherwise
dataset_with_offenses_test = dataset_flag.merge(coded_offenses, how='left', left_on='MOST_SERIOUS_OFFENSE_CODE', right_on='Primary offense code')



In [224]:
dataset_with_offenses_test.sample()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_RECORD_STATUS_CODE,INMATE_ADMIN_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,DISCIPLINARY_INFRACTION_COUNT,NextPrefix,NextStart,NextOffense,new_col,Time_Diff,Recidivate,Primary offense code,Description (if needed),Decided Category,Needed a check?,More lenient,More harsh
7251,0006534,AA,1994-03-30,None,EXPIRED,1994-04-21,1994-09-10,INACTIVE,INACTIVE,1994-04-21,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,BLACK,1967-06-26,NEW JERSEY,UNKNOWN,BORN IN U.S.,NaN,0,NaT,0,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [225]:
# Obs not missing Most Serious Offense
dataset_with_offenses_test['MOST_SERIOUS_OFFENSE_CODE'].notnull().sum()

542311

In [226]:
# Of obs not missing Most Serious Offense, how many are missing our coded value?
dataset_with_offenses_test[dataset_with_offenses_test['MOST_SERIOUS_OFFENSE_CODE'].notnull()]['Primary offense code'].isnull().sum()

28706

In [227]:
28706/542311

0.05293272679329757

In [228]:
dataset_with_offenses_test.groupby('Decided Category').count()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_RECORD_STATUS_CODE,INMATE_ADMIN_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,DISCIPLINARY_INFRACTION_COUNT,NextPrefix,NextStart,NextOffense,new_col,Time_Diff,Recidivate,Primary offense code,Description (if needed),Needed a check?,More lenient,More harsh
Decided Category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,117330,117330,117330,117330,117330,117330,117330,116590,116590,116590,115050,116394,116590,117330,117330,117330,115517,117230,117228,76870,117330,58417,117317,117330,58417,117330,117330,117330,117330,117330,117330
2.0,142126,142126,142126,142126,142126,142126,142126,141253,141253,141253,139435,141033,141253,142126,142126,142126,139673,141994,141996,95697,142126,77104,142109,142126,77104,142126,142126,142126,142126,142126,142126
3.0,187798,187798,187798,187798,187798,187798,187798,186876,186876,186876,184053,186702,186876,187798,187798,187798,179207,187418,187429,137583,187798,78461,187793,187798,78461,187798,187798,187798,187798,187798,187798
4.0,55196,55196,55196,55196,55196,55196,55196,54778,54778,54778,54072,54722,54778,55196,55196,55196,53175,55164,55165,38517,55196,25408,55190,55196,25408,55196,55196,55196,55196,55196,55196
5.0,11155,11155,11155,11155,11155,11155,11155,11099,11099,11099,11003,11078,11099,11155,11155,11155,10519,11154,11154,9864,11155,2591,11152,11155,2591,11155,11155,11155,11155,11155,11155


In [229]:
dataset_with_offenses_test.shape

(850970, 32)

In [230]:
# Step 8 and 9
# Now, merge on coded offenses onto NextOffense, turn Decided Category, More Lenient, and more harsh = 0 if recidivism = 0
dataset_with_offenses = dataset_flag.merge(coded_offenses, how='left', left_on='NextOffense', right_on='Primary offense code')
dataset_with_offenses.loc[dataset_with_offenses['Recidivate']==0,'Decided Category'] = 0
dataset_with_offenses.loc[dataset_with_offenses['Recidivate']==0,'More lenient'] = 0
dataset_with_offenses.loc[dataset_with_offenses['Recidivate']==0,'More harsh'] = 0
dataset_with_offenses.shape

(850970, 32)

In [231]:
dataset_with_offenses.groupby('Decided Category').count()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_RECORD_STATUS_CODE,INMATE_ADMIN_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,DISCIPLINARY_INFRACTION_COUNT,NextPrefix,NextStart,NextOffense,new_col,Time_Diff,Recidivate,Primary offense code,Description (if needed),Needed a check?,More lenient,More harsh
Decided Category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,692709,692709,692709,452527,692709,692709,692709,688110,688110,688110,682473,519274,688110,692709,692708,692709,665212,692031,692046,408973,692709,261928,636906,692709,261928,692709,195361,195361,195361,692709,692709
1.0,21758,21758,21758,19576,21758,21758,21758,21559,21559,21559,21039,21525,21559,21758,21758,21758,21616,21758,21758,18510,21758,21758,21758,21758,21758,21758,21758,21758,21758,21758,21758
2.0,28664,28664,28664,24654,28664,28664,28664,28419,28419,28419,27795,28381,28419,28664,28664,28664,28461,28663,28664,24802,28664,28664,28664,28664,28664,28664,28664,28664,28664,28664,28664
3.0,35608,35608,35608,31709,35608,35608,35608,35391,35391,35391,34291,35358,35391,35608,35608,35608,35128,35599,35602,31968,35608,35608,35608,35608,35608,35608,35608,35608,35608,35608,35608
4.0,9300,9300,9300,8222,9300,9300,9300,9200,9200,9200,8964,9194,9200,9300,9300,9300,9208,9300,9300,8139,9300,9300,9300,9300,9300,9300,9300,9300,9300,9300,9300
5.0,2250,2250,2250,1132,2250,2250,2250,2235,2235,2235,2054,2232,2235,2250,2250,2250,2188,2250,2250,2181,2250,2250,2250,2250,2250,2250,2250,2250,2250,2250,2250


In [232]:
692709 + 21758 + 28664 + 35608 + 9300 + 2250

790289

In [233]:
1-((692709 + 21758 + 28664 + 35608 + 9300 + 2250)/dataset_with_offenses.shape[0])

0.07130803671104735

In [234]:
# Step 10 (next two cells)

# demographic data is not missing in inmate profile, but its missing systematically for ~6000 observations
# in final dataset - which 6k observations are missing in inmate profile?

# based on spot checking many of the 6k obs missing from inmate profile, it appears that NC offender search says
# their incarceration record is currently unavailable - not sure for what reason

# so it makes sense to try and get a lot of these variables from other datasets where possible (e.g. demographics
# from offender profile, disciplinary infractions from the displinary infractions dataset)
inmt_profile.isnull().sum()

ID                                   0
INMATE_RECORD_STATUS_CODE            0
INMATE_ADMIN_STATUS_CODE             0
DATE_OF_LAST_INMATE_MOVEMENT         0
TYPE_OF_LAST_INMATE_MOVEMENT      5774
CURRENT_COMMITMENT_PREFIX       143128
CONTROL_STATUS                       0
dtype: int64

In [235]:
# by bringing in data from other sources, there are a lot fewer missings
# still - need to decide how to deal with features that are missing
dataset_with_offenses.isnull().sum()

ID                                     0
COMMITMENT_PREFIX                      0
EARLIEST_SENTENCE_EFFECTIVE_DT         0
MOST_SERIOUS_OFFENSE_CODE         308659
INMATE_COMPUTATION_STATUS_FLAG         0
END_DATE                               0
PROJ_END_DATE                          0
INMATE_RECORD_STATUS_CODE           6216
INMATE_ADMIN_STATUS_CODE            6216
DATE_OF_LAST_INMATE_MOVEMENT        6216
TYPE_OF_LAST_INMATE_MOVEMENT       15212
CURRENT_COMMITMENT_PREFIX         202165
CONTROL_STATUS                      6216
GENDER                                 0
RACE                                   2
BIRTH_DATE                             0
STATE_BORN                         29626
ETHNICITY                            692
CITIZENSHIP                          669
DISCIPLINARY_INFRACTION_COUNT     310201
NextPrefix                             0
NextStart                         430781
NextOffense                       111635
new_col                                0
Time_Diff       

In [236]:
#dataset_with_offenses.groupby('TYPE_OF_LAST_INMATE_MOVEMENT').count()

In [ ]:
dataset_with_offenses.to_csv('../data/dataset_main.csv', index=False)
active_sentences.to_csv('../data/active_sentences.csv', index=False)